In [8]:
import act.api
import pandas as pd
import requests
import numpy as np

c = act.api.Act("http://35.192.201.254", user_id = 1, log_level = "warning")
#c = act.api.Act("https://act-eu1.mnemonic.no", user_id = 1, log_level = "warning")

In [10]:
df_all=pd.read_excel(r"D:\MY STUFF\rhym\act_api\data\objects_fact_TA.xlsx")
df_infra_all=pd.read_excel(r"D:\MY STUFF\rhym\act_api\data\objects_fact_TA.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\MY STUFF\\rhym\\act_api\\data\\objects_fact_TA.xlsx'

In [5]:
# function to collect unique objects
def all_objects(df):
    all_objects=[]
    source=list(df['source_node'])
    dest=list(df['dest_node'])   
    all_objects.extend(source)
    all_objects.extend(dest)
    all_objects=list(set(all_objects))
    return all_objects

In [74]:
# pusing all objects if not exists in instance
def creating_objects(connection,df,patterns):
    #pattern is a dictionary of object name and its pattern
    objects_exist=[i.name for i in connection.get_object_types()]
    objects=all_objects(df)
    for object_ in objects:
        if ((object_ in patterns) and (object_ not in objects_exist)):
                print(object_,"is pushed to instance")
                c.object_type(object_,patterns[object_]).add()        
        elif ((object_ not in patterns) and (object_ not in objects_exist)):
                print(object_,"is pushed to instance")
                c.object_type(object_).add() 
        elif (object_ in objects_exist):
            print("+++++++------"+object_,"object type already exists in instance"+"-----+++++++")
    print('\n\t\t_______________Task completed___________')

In [75]:
pattern=dict()
creating_objects(connection=c,df=df_all,patterns=pattern)

+++++++------malware object type already exists in instance-----+++++++
+++++++------tool object type already exists in instance-----+++++++
+++++++------attack pattern object type already exists in instance-----+++++++
+++++++------identity object type already exists in instance-----+++++++
+++++++------threat_actor object type already exists in instance-----+++++++
+++++++------campaign object type already exists in instance-----+++++++
+++++++------intrusion_set object type already exists in instance-----+++++++
+++++++------indicator object type already exists in instance-----+++++++
+++++++------vulnerability object type already exists in instance-----+++++++
+++++++------location object type already exists in instance-----+++++++
+++++++------infrastructure object type already exists in instance-----+++++++

		_______________Task completed___________


In [9]:
#optional
# checking weather objects created or not
obj_type_details=dict()
for obj_type in c.get_object_types():
    n=obj_type.name
    i=obj_type.id
    obj_type_details[i]=n
obj_type_details

{'899501e5-8352-4492-bcf9-6d3152c13d64': 'num_files',
 '11536a94-8be2-4a87-b1f5-7cfe6f9d154b': 'tool',
 '5ffa4d2d-50b7-441a-9e8c-484c91b983ec': 'threat_actor',
 '1129bdcb-bfb4-40d9-ae60-935291c66962': 'OS',
 'aff51da0-c9bb-4384-83c9-dd3b4819796a': 'ip',
 'c98d83bd-7e51-4010-a245-cd987f29ce21': 'patch_details',
 'fe059b61-6d9a-4658-877a-db1be6ce7259': 'attack pattern',
 'b5e263e5-6a62-4e17-afb2-15571762ac56': 'num_dlls',
 '781b327b-29ce-475f-b60b-99bfd8b79cbe': 'fqdn',
 'bac5c42b-1466-4909-a9b7-dd774099227e': 'malware',
 '776fee5e-78f5-4af5-9a53-616610c46c1f': 'infrastructure',
 'c6a46e2d-f57c-4b4c-a695-b309b057b8ae': 'malicious_file',
 '2a3f6a6d-1215-4860-9306-8baf20e6ba70': 'intrusion_set',
 '85cf8b35-f52b-46e8-8f64-2618b7ceaf9f': 'num_cpls',
 '6e2a3c16-6c88-47b9-8e63-670ab26ec82c': 'location',
 '28edcfdd-2371-4566-88ae-0fbe58ab52ad': 'observed data',
 '54fcb681-fdbc-4772-ab65-df74d3f25e91': 'dotnet_framework_version',
 '0428de17-e23f-4781-9cbb-b6a617767fbb': 'size',
 '384f595e-e37b-4

In [359]:
def create_facts(connection,df):

    for index,row in df.iterrows():
            if row['Bidirectional']==1:
                object_bindings = [{
                "destinationObjectType": row['dest_node'],
                "sourceObjectType": row['source_node'],
                "bidirectional":True}]
            elif row['Bidirectional']==0:
                object_bindings = [{
                "destinationObjectType": row['dest_node'],
                "sourceObjectType": row['source_node']}]
            else:
                print(row['Bidirectional']+' is unexpected at '+row['fact'])
                
        
            c.create_fact_type(row['fact'], object_bindings = object_bindings)

            
    print('\n\t\t_____task done_______')
    

NameError: name 'df' is not defined

In [360]:
create_facts(connection=c,df=df)

[2020-04-22 17:53:29] app=act level=WARNING msg=Fact type uses already exists
[2020-04-22 17:53:29] app=act level=WARNING msg=All bindings specified for uses already exists
[2020-04-22 17:53:29] app=act level=WARNING msg=Fact type uses already exists
[2020-04-22 17:53:29] app=act level=WARNING msg=All bindings specified for uses already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=Fact type authored by already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=All bindings specified for authored by already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=Fact type attributed to already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=All bindings specified for attributed to already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=Fact type impersonates already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=All bindings specified for impersonates already exists
[2020-04-22 17:53:30] app=act level=WARNING msg=Fact type targets already exists
[2020-

In [14]:
for i in c.get_fact_types():
    if i.name=='uses':
        print(i)

FactType(name='uses', id='0868cb46-ff7d-49d4-a132-a1a92a4feab4', default_confidence=1.0, relevant_object_bindings=[RelevantObjectBindings(source_object_type='f86b46ab-064b-44f3-9264-44274e55ff93', destination_object_type='b872a367-5b85-410d-9133-2a7701f3c4c6'), RelevantObjectBindings(source_object_type='f86b46ab-064b-44f3-9264-44274e55ff93', destination_object_type='41a76fb6-3eb7-4fcc-8647-112326f9a4dd'), RelevantObjectBindings(source_object_type='f86b46ab-064b-44f3-9264-44274e55ff93', destination_object_type='99a99af5-0ae1-4486-a861-9fd033702014'), RelevantObjectBindings(source_object_type='f86b46ab-064b-44f3-9264-44274e55ff93', destination_object_type='8b9f1a14-7379-44cc-ac31-ff266b328d9e'), RelevantObjectBindings(source_object_type='f86b46ab-064b-44f3-9264-44274e55ff93', destination_object_type='a70b0efd-4eb2-4145-a38f-c97e00440318'), RelevantObjectBindings(source_object_type='8b9f1a14-7379-44cc-ac31-ff266b328d9e', destination_object_type='b872a367-5b85-410d-9133-2a7701f3c4c6')], na

In [13]:
df_ta_mal=pd.read_csv(r"D:\MY STUFF\rhym\act_api\data\object_facts_values\threat_actor-malware.csv")
df_ta_ap=pd.read_csv(r"D:\MY STUFF\rhym\act_api\data\object_facts_values\threat_actor-attack_pattern.csv")
df_ta_tool=pd.read_csv(r"D:\MY STUFF\rhym\act_api\data\object_facts_values\threat_actor-tools.csv")
df_mal_ta=pd.read_csv(r"D:\MY STUFF\rhym\act_api\data\object_facts_values\malware-threat_actor.csv")
df_ta_vul=pd.read_csv(r"D:\MY STUFF\rhym\act_api\data\object_facts_values\threat_actor_vulnerability.csv")

In [14]:
def push_object_values_to_facts(connection,df_values,df_all):
    
    col=df_values.columns
    relation=df_all['fact'][(df_all['source_node']==col[0])&(df_all['dest_node']==col[1])]
    if len(relation)!=0:
        relation=list(relation)[0]
        print(relation)
        for index,row in df_values.iterrows():
            connection.fact(relation).source(col[0], row[col[0]]).destination(col[1], row[col[1]]).add()
            print(index)
    elif len(relation)==0:
            print('fact type does not exist between',col[0]+' and '+col[1])
    
    print('\n\t\t________taskdone______________')
    

In [15]:
#push_object_values_to_facts(connection=c,df_values=df_ta_mal,df_all=df_all)
push_object_values_to_facts(connection=c,df_values=df_mal_ta,df_all=df_all)

NameError: name 'df_all' is not defined

In [90]:
c.object_search(object_value="Mimikatz")

[Object(type='tool', value='Mimikatz', id='37730365-14fd-4f34-b3a5-e99cfe500dd1')]

In [92]:
obj = c.object("threat_actor", "apt1").traverse('g.outE()')
print(obj)

[Fact(type='uses', origin=Origin(name='John Doe', id='00000000-0000-0000-0000-000000000001'), confidence=1.0, organization=Organization(name='Test Organization 1', id='00000000-0000-0000-0000-000000000001'), source_object=Object(type='threat_actor', value='apt1', id='82862eb6-a925-47db-b002-8014383b88c9'), destination_object=Object(type='tool', value='xCmd', id='7016c6ea-0154-485b-9791-6e3fcce70e29')), Fact(type='uses', origin=Origin(name='John Doe', id='00000000-0000-0000-0000-000000000001'), confidence=1.0, organization=Organization(name='Test Organization 1', id='00000000-0000-0000-0000-000000000001'), source_object=Object(type='threat_actor', value='apt1', id='82862eb6-a925-47db-b002-8014383b88c9'), destination_object=Object(type='malware', value='Seasalt', id='e933955b-a87f-4848-b7b5-755bb8d496a0')), Fact(type='uses', origin=Origin(name='John Doe', id='00000000-0000-0000-0000-000000000001'), confidence=1.0, organization=Organization(name='Test Organization 1', id='00000000-0000-00

In [ ]:
for i in c.get_fact_types():
    if i.name=='uses':
        print(i.relevant_fact_bindings)

In [191]:
df=pd.DataFrame(obj_type_details.items(),columns=['UUID','Object_name'])
df.to_csv(r'D:\MY STUFF\rhym\act_api\data\objects.csv',index=None)

In [2]:
c.get_fact_types()

[FactType(name='controls', id='fe6d4918-1a60-4609-bfa3-5341a4dfd92f', default_confidence=1.0, relevant_object_bindings=[RelevantObjectBindings(source_object_type='776fee5e-78f5-4af5-9a53-616610c46c1f', destination_object_type='bac5c42b-1466-4909-a9b7-dd774099227e'), RelevantObjectBindings(source_object_type='bac5c42b-1466-4909-a9b7-dd774099227e', destination_object_type='bac5c42b-1466-4909-a9b7-dd774099227e')], namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), FactType(name='category', id='287673ed-22e5-40ab-b99c-1ff74a5cab5b', default_confidence=1.0, validator='TrueValidator', validator_parameter=None, relevant_object_bindings=[RelevantObjectBindings(source_object_type='aff51da0-c9bb-4384-83c9-dd3b4819796a', destination_object_type='781b327b-29ce-475f-b60b-99bfd8b79cbe', bidirectional_binding=True)], namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), FactType(name='exploits', id='58c1c70d-18d9-4cde-a5ca-96e2954e267c', default_

In [7]:
for i in c.get_fact_types():
    if i.name=='has':
        print(i)

FactType(name='has', id='30790683-d76f-4bfe-bee7-e923c144f679', default_confidence=1.0, relevant_object_bindings=[RelevantObjectBindings(source_object_type='9cc89b6e-c104-44b2-9605-28ef5f544387', destination_object_type='899501e5-8352-4492-bcf9-6d3152c13d64'), RelevantObjectBindings(source_object_type='e61a959a-2dc6-45e1-895d-c8db592b3abc', destination_object_type='b5e263e5-6a62-4e17-afb2-15571762ac56'), RelevantObjectBindings(source_object_type='e61a959a-2dc6-45e1-895d-c8db592b3abc', destination_object_type='85cf8b35-f52b-46e8-8f64-2618b7ceaf9f'), RelevantObjectBindings(source_object_type='4c8887b9-56f3-43e7-bc6f-698bb2ffe398', destination_object_type='1129bdcb-bfb4-40d9-ae60-935291c66962'), RelevantObjectBindings(source_object_type='4c8887b9-56f3-43e7-bc6f-698bb2ffe398', destination_object_type='54fcb681-fdbc-4772-ab65-df74d3f25e91'), RelevantObjectBindings(source_object_type='11536a94-8be2-4a87-b1f5-7cfe6f9d154b', destination_object_type='3054408b-1d72-4318-bb82-0121e11b5a14'), Rele

In [163]:
relations=[]
for fact_type in c.get_fact_types():
    try:
        fact_name=fact_type.name
        for i in fact_type.relevant_object_bindings:
            relations.append([i.source_object_type.id,i.source_object_type.name,fact_name,i.destination_object_type.name,i.destination_object_type.id])
    except:
        print(fact_type.name)
        pass

observationTime
Retraction


In [179]:
df=pd.DataFrame(relations,columns=['source_id','source_name','relation','dest_name','dest_id'])
df.to_csv(r'D:\MY STUFF\rhym\act_api\data\facts.csv')

In [203]:
c.get_fact_types()

[FactType(name='resolve', id='5ff1ef61-9ed4-4139-a6ff-956252ad9ee0', default_confidence=1.0, validator='TrueValidator', validator_parameter=None, relevant_object_bindings=[RelevantObjectBindings(source_object_type='b8c8878e-30da-49c4-a49e-f0125f9e7451', destination_object_type='50d0a570-3994-4a8f-915c-66e5df6aa7e3', bidirectional_binding=True)], namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), FactType(name='resolves', id='0dafa0c3-9672-45b7-b8ba-cf5850d6bfe0', default_confidence=1.0, validator='TrueValidator', validator_parameter=None, relevant_object_bindings=[RelevantObjectBindings(source_object_type='b8c8878e-30da-49c4-a49e-f0125f9e7451', destination_object_type='50d0a570-3994-4a8f-915c-66e5df6aa7e3')], namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000'))]

In [46]:
c.get_object_types()

[ObjectType(name='indicator', id='5570544b-f1b1-4238-b0ef-376f71a2b5ef', namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), ObjectType(name='attack pattern', id='41a76fb6-3eb7-4fcc-8647-112326f9a4dd', namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), ObjectType(name='infrastructure', id='b872a367-5b85-410d-9133-2a7701f3c4c6', namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), ObjectType(name='intrusion_set', id='a5a0c968-e1a6-48bb-94b3-ead34ab668b5', namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), ObjectType(name='ip', id='b8c8878e-30da-49c4-a49e-f0125f9e7451', validator='TrueValidator', validator_parameter=None, namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), ObjectType(name='threat_actor', id='f86b46ab-064b-44f3-9264-44274e55ff93', namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000')), ObjectType(name='campaig

In [213]:
resolves = [ft for ft in c.get_fact_types() if ft.name == "resolves"][0]

In [211]:
object_bindings = [{
    "destinationObjectType": 'abcd',
    "sourceObjectType": "ipv4"
}]
c.create_fact_type("checking", object_bindings = object_bindings)

FactType(name='checking', id='d898346f-5cf2-450f-a412-1e41345d51e9', default_confidence=1.0, relevant_object_bindings=[RelevantObjectBindings(source_object_type='c4638e84-3509-49d0-b9a3-e32e76198254', destination_object_type='48e9699c-8895-4d16-a5e6-ae1c32f96192')], namespace=NameSpace(name='Global', id='00000000-0000-0000-0000-000000000000'))

In [204]:
f = c.fact("resolve").source("ip", "127.1.1.3").destination("fqdn", "github.com").add()

In [ ]:
c.object_search

In [81]:
c.object_search(object_value="Mimikatz")

[Object(type='tool', value='Mimikatz', id='37730365-14fd-4f34-b3a5-e99cfe500dd1')]

In [84]:
obj = c.object("malware", "Yahoyah").traverse('g.outE()')
print(obj)

[Fact(type='authored by', origin=Origin(name='John Doe', id='00000000-0000-0000-0000-000000000001'), confidence=1.0, organization=Organization(name='Test Organization 1', id='00000000-0000-0000-0000-000000000001'), source_object=Object(type='malware', value='Yahoyah', id='80771181-c09d-488f-8c07-8609b0317406'), destination_object=Object(type='threat_actor', value='tropic trooper', id='8c2f6744-53b4-4fe6-9b72-a2337c3db88f'))]
